# **First part**

**1. What is the level of observation?**

   The data is aggregated at the state level.

**2. Construct 1 dummy variable called “post_china” where post_china=1 for year>=2001 and 0 otherwise.**

In [68]:
import pandas as pd
dcbp = pd.read_csv("..\data\datacbp.csv")  
print(dcbp.head())


   fipstate   naics   emp    qp1      ap   est  year
0         1  113///  8207  44757  195720  1082  1998
1         1  114///    93    363    1532    30  1998
2         1  115///  1614  10181   45519   194  1998
3         1  211///   804   9477   38146    38  1998
4         1  212///  7074  79136  318295   159  1998


<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_19868\3337931969.py:2: SyntaxWarning: invalid escape sequence '\d'
  dcbp = pd.read_csv("..\data\datacbp.csv")


In [69]:
dcbp["post_china"]=(dcbp["year"]>= 2001).astype(int)

print(dcbp.head())

   fipstate   naics   emp    qp1      ap   est  year  post_china
0         1  113///  8207  44757  195720  1082  1998           0
1         1  114///    93    363    1532    30  1998           0
2         1  115///  1614  10181   45519   194  1998           0
3         1  211///   804   9477   38146    38  1998           0
4         1  212///  7074  79136  318295   159  1998           0


In [70]:
dcbp.drop(dcbp[(dcbp['emp'] == 0) | (dcbp['est'] == 0) | (dcbp['qp1'] == 0)].index, inplace=True)

**3. Construct 1 dummy variable called “manuf” where manuf=1 for all the observations that start with naics code “3” – which is manufacturing - and 0 otherwise.**

In [71]:
dcbp["manuf"]=dcbp["naics"].str.startswith('3').astype(int)
print(dcbp.head())

   fipstate   naics   emp    qp1      ap   est  year  post_china  manuf
0         1  113///  8207  44757  195720  1082  1998           0      0
1         1  114///    93    363    1532    30  1998           0      0
2         1  115///  1614  10181   45519   194  1998           0      0
3         1  211///   804   9477   38146    38  1998           0      0
4         1  212///  7074  79136  318295   159  1998           0      0


**4. Construct the values necessary to generate the difference-in-difference estimate (i.e. 2x2 Matrix) of the effect of China entering the WTO on employment (emp).**

In [72]:
# Mean of controls before treatment
mean_bef_non_manuf = dcbp.loc[(dcbp['manuf'] == 0) & (dcbp['post_china'] == 0), 'emp'].mean()
print(mean_bef_non_manuf)

# Mean of controls after treatment
mean_aft_non_manuf = dcbp.loc[(dcbp['manuf'] == 0) & (dcbp['post_china'] == 1), 'emp'].mean()
print(mean_aft_non_manuf)

# Mean of treat before treatment
mean_bef_manuf = dcbp.loc[(dcbp['manuf'] == 1) & (dcbp['post_china'] == 0), 'emp'].mean()
print(mean_bef_manuf)

# Mean of treat after treatment
mean_aft_manuf = dcbp.loc[(dcbp['manuf'] == 1) & (dcbp['post_china'] == 1), 'emp'].mean()
print(mean_aft_manuf)

# Finally
result = (mean_bef_non_manuf - mean_aft_non_manuf) - (mean_bef_manuf - mean_aft_manuf)
print("The result is", result)


35231.096024387145
36021.81903276131
17872.79878267096
15043.595137420718
The result is -3619.9266536244104


**Interpretation**
The results show that China's entry into the WTO in 2001 had a significant negative impact on manufacturing employment in the US, with an estimated reduction of 3619 jobs in the sector.

**5. Estimate a diff-in-diff regression and make sure you get the same diff-in-diff estimate as in part 4.**

In [73]:

#!pip install statsmodels
import statsmodels.formula.api as smf

# Especificamos el modelo de Diferencias en Diferencias (DiD)
model = smf.ols("emp ~ post_china + manuf + post_china:manuf", data=dcbp).fit()

# Mostramos los resultados de la regresión
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                    emp   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     205.3
Date:               mar, 04 Feb. 2025   Prob (F-statistic):          6.40e-132
Time:                        14:51:11   Log-Likelihood:            -4.1187e+05
No. Observations:               33008   AIC:                         8.238e+05
Df Residuals:                   33004   BIC:                         8.238e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept         3.523e+04    715.864  

**6. Estimate a diff-in-diff regression for the effect of China entering the WTO in 2001 on the number of establishment (est), an average pay (qp1/emp). Interpret the results.**


In [74]:
# DID for number of establishment
model = smf.ols("est ~ post_china + manuf + post_china:manuf", data=dcbp).fit()

# Mostramos los resultados de la regresión
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    est   R-squared:                       0.040
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     458.5
Date:               mar, 04 Feb. 2025   Prob (F-statistic):          7.09e-292
Time:                        14:51:12   Log-Likelihood:            -3.2465e+05
No. Observations:               33008   AIC:                         6.493e+05
Df Residuals:                   33004   BIC:                         6.493e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept         2472.6250     50.963  

**Interpretation**
There is no significant effect of China's entry into the WTO on the number of manufacturing establishments at the 5% significance level.

In [75]:
# We create "average pay"
dcbp["average_pay"]=dcbp["qp1"]/dcbp["emp"]

# DID for average pay
model = smf.ols("average_pay ~ post_china + manuf + post_china:manuf", data=dcbp).fit()

# Mostramos los resultados de la regresión
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            average_pay   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     241.7
Date:               mar, 04 Feb. 2025   Prob (F-statistic):          3.76e-155
Time:                        14:51:12   Log-Likelihood:            -1.0015e+05
No. Observations:               33008   AIC:                         2.003e+05
Df Residuals:                   33004   BIC:                         2.003e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            7.1341      0.057  

**Interpretation**
There is no significant effect of China's entry into the WTO on the quarterly wage of workers in manufacturing firms at the 5% significance level.

**7.Estimate same regression as in (5) but now take logs of the dependent variable (i,e, log(emp)). Interpret your results. Is it necessary to take logs?**

In [76]:
import numpy as np

# Crear la columna con el logaritmo de 'emp'
dcbp['log_emp'] = np.log(dcbp['emp'])


# Especificamos el modelo de Diferencias en Diferencias (DiD)
model = smf.ols("log_emp ~ post_china + manuf + post_china:manuf", data=dcbp).fit()

# Mostramos los resultados de la regresión
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                log_emp   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     298.2
Date:               mar, 04 Feb. 2025   Prob (F-statistic):          5.15e-191
Time:                        14:51:12   Log-Likelihood:                -63164.
No. Observations:               33008   AIC:                         1.263e+05
Df Residuals:                   33004   BIC:                         1.264e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            9.3691      0.018  

**Interpretation:** The distribution of "emp" is skewed to the left and applying a logarithmic transformation stabilizes the variance, which improves the model fit. About the interpretation, manufacturing jobs in the US decreased by approximately 16.72% compared to other sectors or periods.

**8.Generate one dummy per year. Construct the interaction between each year dummies and your treatment group (manuf). You should have 9 interaction terms**

In [77]:
# Generar dummies para cada año
year_dummies = pd.get_dummies(dcbp['year'], prefix='year')

# Añadir las dummies al dataframe original
dcbp = pd.concat([dcbp, year_dummies], axis=1)

# Crear interacciones entre las dummies de año y el grupo de tratamiento (manuf)
for year_col in year_dummies.columns:
    dcbp[f'{year_col}_manuf'] = dcbp[year_col] * dcbp['manuf']

# Verificar las nuevas columnas generadas
print(dcbp.head())


   fipstate   naics   emp    qp1      ap   est  year  post_china  manuf  \
0         1  113///  8207  44757  195720  1082  1998           0      0   
1         1  114///    93    363    1532    30  1998           0      0   
2         1  115///  1614  10181   45519   194  1998           0      0   
3         1  211///   804   9477   38146    38  1998           0      0   
4         1  212///  7074  79136  318295   159  1998           0      0   

   average_pay  ...  year_2006  year_1998_manuf  year_1999_manuf  \
0     5.453515  ...      False                0                0   
1     3.903226  ...      False                0                0   
2     6.307931  ...      False                0                0   
3    11.787313  ...      False                0                0   
4    11.186882  ...      False                0                0   

   year_2000_manuf  year_2001_manuf  year_2002_manuf  year_2003_manuf  \
0                0                0                0               

**9. Estimate an event study, i.e. run the following specification: log(emp) vs year dummies, manuf*year dummies (omit the interaction between manuf  * year 1998) and control for NAICS-3 digit dummies and state dummies. Interpret your results. Should you expect to see any effect for the interaction term manuf*year 1999 or manuf * year 2000? Did the China shock have a significant effect on employment? Was it a short-run or long-run effect?**

In [83]:
# Crear dummies para NAICS de 3 dígitos
dcbp['naics_3'] = dcbp['naics'].str[:3]
naics_dummies = pd.get_dummies(dcbp['naics_3'], prefix='naics_3')

# Crear dummies para la columna 'fipstate'
state_dummies = pd.get_dummies(dcbp['fipstate'], prefix='state')

# Combinar las dummies con el dataframe original
dcbp = pd.concat([dcbp, naics_dummies, state_dummies], axis=1)

In [84]:
# Convertir todas las columnas booleanas a enteros
bool_columns = dcbp.select_dtypes(include=['bool']).columns
dcbp[bool_columns] = dcbp[bool_columns].astype(int)


In [85]:
# Eliminar columnas duplicadas
dcbp = dcbp.loc[:, ~dcbp.columns.duplicated()]

In [97]:
# Filtrar solo las columnas de dummies de años (que NO terminan en '_manuf')
year_columns = [col for col in dcbp.columns if col.startswith('year_') and not col.endswith('_manuf')]

# Filtrar solo las columnas de interacciones (terminan en '_manuf')
year_manuf_columns = [col for col in dcbp.columns if col.endswith('_manuf')]

# Filtrar las columnas de NAICS-3 y estados
naics_columns = [col for col in dcbp.columns if col.startswith('naics_3_')]
state_columns = [col for col in dcbp.columns if col.startswith('state_')]

# Crear la fórmula de regresión
formula = "log_emp ~ " + \
          " + ".join([col for col in dcbp.columns if col.startswith('year_') and not col.endswith('_manuf') and col != 'year_1998']) + " + " + \
          " + ".join([col for col in dcbp.columns if col.endswith('_manuf') and col != 'year_1998_manuf']) + " + " + \
          " + ".join([col for col in dcbp.columns if col.startswith('naics_3_') and col != 'naics_3_111']) + " + " + \
          " + ".join([col for col in dcbp.columns if col.startswith('state_') and col != 'state_1'])

# Verificar la fórmula generada
print("Fórmula generada:")
print(formula)

# Ajustar el modelo
import statsmodels.formula.api as smf

model = smf.ols(formula, data=dcbp).fit()

# Mostrar los resultados
print(model.summary())


Fórmula generada:
log_emp ~ year_1999 + year_2000 + year_2001 + year_2002 + year_2003 + year_2004 + year_2005 + year_2006 + year_1999_manuf + year_2000_manuf + year_2001_manuf + year_2002_manuf + year_2003_manuf + year_2004_manuf + year_2005_manuf + year_2006_manuf + naics_3_113 + naics_3_114 + naics_3_115 + naics_3_211 + naics_3_212 + naics_3_213 + naics_3_221 + naics_3_233 + naics_3_234 + naics_3_235 + naics_3_236 + naics_3_237 + naics_3_238 + naics_3_311 + naics_3_312 + naics_3_313 + naics_3_314 + naics_3_315 + naics_3_316 + naics_3_321 + naics_3_322 + naics_3_323 + naics_3_324 + naics_3_325 + naics_3_326 + naics_3_327 + naics_3_331 + naics_3_332 + naics_3_333 + naics_3_334 + naics_3_335 + naics_3_336 + naics_3_337 + naics_3_339 + naics_3_421 + naics_3_422 + naics_3_423 + naics_3_424 + naics_3_425 + naics_3_441 + naics_3_442 + naics_3_443 + naics_3_444 + naics_3_445 + naics_3_446 + naics_3_447 + naics_3_448 + naics_3_451 + naics_3_452 + naics_3_453 + naics_3_454 + naics_3_481 + naic

In [105]:
# Extraer los coeficientes y asociarlos a sus variables
coeficientes_df = pd.DataFrame({
    "Variable": model.params.index,  # Nombres de las variables
    "Coeficiente": model.params.values,  # Valores de los coeficientes
    "Std. Error": model.bse.values,  # Errores estándar
    "t-Statistic": model.tvalues.values,  # Estadísticos t
    "P-Value": model.pvalues.values  # Valores p
})

print(coeficientes_df.loc[9:16])



           Variable  Coeficiente  Std. Error  t-Statistic   P-Value
9   year_1999_manuf    -0.019211    0.079855    -0.240566  0.809893
10  year_2000_manuf    -0.080266    0.079636    -1.007910  0.313505
11  year_2001_manuf    -0.074140    0.079478    -0.932833  0.350913
12  year_2002_manuf    -0.047060    0.079619    -0.591062  0.554483
13  year_2003_manuf    -0.161241    0.079198    -2.035919  0.041766
14  year_2004_manuf    -0.136106    0.079511    -1.711779  0.086947
15  year_2005_manuf    -0.170370    0.079375    -2.146412  0.031847
16  year_2006_manuf    -0.185321    0.079392    -2.334263  0.019588


**Interpretation:** The regression results suggest that the interaction terms manuf * year 1999 and manuf * year 2000 are not statistically significant, as indicated by their high p-values. This implies that there is no evidence of a measurable effect on U.S. manufacturing employment from the China shock in the immediate pre-WTO accession period. About second question, Based on the regression results, the interaction terms for manuf with years 2003 through 2006  are statistically significant, as their p-values are very small (below 0.05). This indicates that the China shock had a significant and negative effect on U.S. manufacturing employment in the long run, particularly after 2003. In contrast, earlier years, such as 2001 and 2002, show coefficients that are not statistically significant, suggesting that the short-run impact of the China shock was less pronounced.

**10.	[4 points ] Estimate a similar event study on the log(est) and average pay. Interpret your results.**

## Log est

In [99]:

# Crear la columna con el logaritmo de 'est'
dcbp['log_est'] = np.log(dcbp['est'])

# Filtrar solo las columnas de dummies de años (que NO terminan en '_manuf')
year_columns = [col for col in dcbp.columns if col.startswith('year_') and not col.endswith('_manuf')]

# Filtrar solo las columnas de interacciones (terminan en '_manuf')
year_manuf_columns = [col for col in dcbp.columns if col.endswith('_manuf')]

# Filtrar las columnas de NAICS-3 y estados
naics_columns = [col for col in dcbp.columns if col.startswith('naics_3_')]
state_columns = [col for col in dcbp.columns if col.startswith('state_')]

# Crear la fórmula de regresión
formula = "log_est ~ " + \
          " + ".join([col for col in dcbp.columns if col.startswith('year_') and not col.endswith('_manuf') and col != 'year_1998']) + " + " + \
          " + ".join([col for col in dcbp.columns if col.endswith('_manuf') and col != 'year_1998_manuf']) + " + " + \
          " + ".join([col for col in dcbp.columns if col.startswith('naics_3_') and col != 'naics_3_111']) + " + " + \
          " + ".join([col for col in dcbp.columns if col.startswith('state_') and col != 'state_1'])

# Verificar la fórmula generada
print("Fórmula generada:")
print(formula)

# Ajustar el modelo
import statsmodels.formula.api as smf

model = smf.ols(formula, data=dcbp).fit()

# Mostrar los resultados
print(model.summary())

Fórmula generada:
log_est ~ year_1999 + year_2000 + year_2001 + year_2002 + year_2003 + year_2004 + year_2005 + year_2006 + year_1999_manuf + year_2000_manuf + year_2001_manuf + year_2002_manuf + year_2003_manuf + year_2004_manuf + year_2005_manuf + year_2006_manuf + naics_3_113 + naics_3_114 + naics_3_115 + naics_3_211 + naics_3_212 + naics_3_213 + naics_3_221 + naics_3_233 + naics_3_234 + naics_3_235 + naics_3_236 + naics_3_237 + naics_3_238 + naics_3_311 + naics_3_312 + naics_3_313 + naics_3_314 + naics_3_315 + naics_3_316 + naics_3_321 + naics_3_322 + naics_3_323 + naics_3_324 + naics_3_325 + naics_3_326 + naics_3_327 + naics_3_331 + naics_3_332 + naics_3_333 + naics_3_334 + naics_3_335 + naics_3_336 + naics_3_337 + naics_3_339 + naics_3_421 + naics_3_422 + naics_3_423 + naics_3_424 + naics_3_425 + naics_3_441 + naics_3_442 + naics_3_443 + naics_3_444 + naics_3_445 + naics_3_446 + naics_3_447 + naics_3_448 + naics_3_451 + naics_3_452 + naics_3_453 + naics_3_454 + naics_3_481 + naic

In [106]:
# Extraer los coeficientes y asociarlos a sus variables
coeficientes_df2 = pd.DataFrame({
    "Variable": model.params.index,  # Nombres de las variables
    "Coeficiente": model.params.values,  # Valores de los coeficientes
    "Std. Error": model.bse.values,  # Errores estándar
    "t-Statistic": model.tvalues.values,  # Estadísticos t
    "P-Value": model.pvalues.values  # Valores p
})

print(coeficientes_df2.loc[9:16])

           Variable  Coeficiente  Std. Error  t-Statistic   P-Value
9   year_1999_manuf    -0.019211    0.079855    -0.240566  0.809893
10  year_2000_manuf    -0.080266    0.079636    -1.007910  0.313505
11  year_2001_manuf    -0.074140    0.079478    -0.932833  0.350913
12  year_2002_manuf    -0.047060    0.079619    -0.591062  0.554483
13  year_2003_manuf    -0.161241    0.079198    -2.035919  0.041766
14  year_2004_manuf    -0.136106    0.079511    -1.711779  0.086947
15  year_2005_manuf    -0.170370    0.079375    -2.146412  0.031847
16  year_2006_manuf    -0.185321    0.079392    -2.334263  0.019588


The results show that the number of manufacturing establishments decreased significantly in the long term due to the China shock. For example, the coefficient for `year_2006_manuf` is -0.185, indicating an approximate 18.5% decline in the number of manufacturing establishments by 2006 relative to the baseline. Significant effects begin in 2003 (16.1% decrease, p = 0.0418) and continue through 2006 (p = 0.0196), while earlier years show no statistically significant changes. This highlights a gradual, cumulative impact on manufacturing establishments over time.

## Average pay

In [108]:
# Filtrar solo las columnas de dummies de años (que NO terminan en '_manuf')
year_columns = [col for col in dcbp.columns if col.startswith('year_') and not col.endswith('_manuf')]

# Filtrar solo las columnas de interacciones (terminan en '_manuf')
year_manuf_columns = [col for col in dcbp.columns if col.endswith('_manuf')]

# Filtrar las columnas de NAICS-3 y estados
naics_columns = [col for col in dcbp.columns if col.startswith('naics_3_')]
state_columns = [col for col in dcbp.columns if col.startswith('state_')]

# Crear la fórmula de regresión
formula = "average_pay ~ " + \
          " + ".join([col for col in dcbp.columns if col.startswith('year_') and not col.endswith('_manuf') and col != 'year_1998']) + " + " + \
          " + ".join([col for col in dcbp.columns if col.endswith('_manuf') and col != 'year_1998_manuf']) + " + " + \
          " + ".join([col for col in dcbp.columns if col.startswith('naics_3_') and col != 'naics_3_111']) + " + " + \
          " + ".join([col for col in dcbp.columns if col.startswith('state_') and col != 'state_1'])

# Verificar la fórmula generada
print("Fórmula generada:")
print(formula)

# Ajustar el modelo
import statsmodels.formula.api as smf

model = smf.ols(formula, data=dcbp).fit()

# Mostrar los resultados
print(model.summary())

Fórmula generada:
average_pay ~ year_1999 + year_2000 + year_2001 + year_2002 + year_2003 + year_2004 + year_2005 + year_2006 + year_1999_manuf + year_2000_manuf + year_2001_manuf + year_2002_manuf + year_2003_manuf + year_2004_manuf + year_2005_manuf + year_2006_manuf + naics_3_113 + naics_3_114 + naics_3_115 + naics_3_211 + naics_3_212 + naics_3_213 + naics_3_221 + naics_3_233 + naics_3_234 + naics_3_235 + naics_3_236 + naics_3_237 + naics_3_238 + naics_3_311 + naics_3_312 + naics_3_313 + naics_3_314 + naics_3_315 + naics_3_316 + naics_3_321 + naics_3_322 + naics_3_323 + naics_3_324 + naics_3_325 + naics_3_326 + naics_3_327 + naics_3_331 + naics_3_332 + naics_3_333 + naics_3_334 + naics_3_335 + naics_3_336 + naics_3_337 + naics_3_339 + naics_3_421 + naics_3_422 + naics_3_423 + naics_3_424 + naics_3_425 + naics_3_441 + naics_3_442 + naics_3_443 + naics_3_444 + naics_3_445 + naics_3_446 + naics_3_447 + naics_3_448 + naics_3_451 + naics_3_452 + naics_3_453 + naics_3_454 + naics_3_481 + 

In [109]:
# Extraer los coeficientes y asociarlos a sus variables
coeficientes_df2 = pd.DataFrame({
    "Variable": model.params.index,  # Nombres de las variables
    "Coeficiente": model.params.values,  # Valores de los coeficientes
    "Std. Error": model.bse.values,  # Errores estándar
    "t-Statistic": model.tvalues.values,  # Estadísticos t
    "P-Value": model.pvalues.values  # Valores p
})

print(coeficientes_df2.loc[9:16])

           Variable  Coeficiente  Std. Error  t-Statistic   P-Value
9   year_1999_manuf    -0.063903    0.230952    -0.276693  0.782018
10  year_2000_manuf    -0.092392    0.230318    -0.401149  0.688313
11  year_2001_manuf    -0.226194    0.229861    -0.984047  0.325100
12  year_2002_manuf    -0.058282    0.230269    -0.253106  0.800188
13  year_2003_manuf     0.032085    0.229051     0.140078  0.888600
14  year_2004_manuf     0.037892    0.229956     0.164780  0.869119
15  year_2005_manuf    -0.048888    0.229561    -0.212962  0.831358
16  year_2006_manuf    -0.134251    0.229610    -0.584688  0.558761


The results indicate that the coefficients for the interaction terms across all years are not statistically significant (p-values > 0.05). This suggests that the China shock did not have a significant impact on average wages in manufacturing over the analyzed period. 